In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv(r'dataset/train.csv')

# Create directories for storing files
import os
if not os.path.exists('image_links'):
    os.makedirs('image_links')

# Extract unique entity names
unique_entities = df['entity_name'].unique()

for entity in unique_entities:
    # Filter by entity_name and sample 100 image links
    filtered_df = df[df['entity_name'] == entity]
    sampled_df = filtered_df.sample(n=100, random_state=42)  # Sample 100 random rows

    # Save image links and entity values
    sampled_df[['image_link']].to_csv(f'image_links/{entity}_image_links.csv', index=False)
    sampled_df[['entity_value', 'image_link']].to_csv(f'image_links/{entity}_entity_value_links.csv', index=False)



from lmdeploy import pipeline, TurbomindEngineConfig
from lmdeploy.vl import load_image
import pandas as pd
import os

# Define your model and configuration with the system prompt
model = 'OpenGVLab/InternVL2-4B'
offload_folder = 'model_offload'
os.makedirs(offload_folder, exist_ok=True)

# Update the configuration with the offload folder
config = TurbomindEngineConfig(
    session_len=8192,
    temperature=0.7,
    system_prompt=(
        "You are a multimodal chatbot capable of analyzing both visual and textual content from images. "
        "Your task is to extract relevant information by:\n\n"
        "Image Text Analysis: Use OCR to extract any text present within the image.\n"
        "Visual Cue Analysis: Identify any additional visual cues such as logos, colors, objects, or shapes that could assist in answering the query.\n\n"
        "After processing both the text and the visual elements:\n\n"
        "If the extracted data provides enough context to respond meaningfully, answer the query with the values with no explanation of your reasoning.\n"
        "If no relevant data is found or the query cannot be answered based on the image content, simply return value 0."
    ),
    offload_folder=offload_folder  # Specify the offload folder
)

try:
    pipe = pipeline(model, backend_config=config)
except Exception as e:
    print(f"An error occurred while creating the pipeline: {e}")
    raise

# Function to get predictions
def get_prediction(image_link, entity_name):
    image = load_image(image_link)
    query = f"What is the {entity_name} of the product?"
    response = pipe((query, image))

    # Extract only the value from the response
    return response.text.strip()

# Process each entity_name
unique_entities = ['weight', 'wattage', 'depth', 'height', 'width', 'voltage', 'maximum_weight_recommendation', 'volume']  # Replace with actual unique entity names

for entity in unique_entities:
    # Load the CSV files
    df = pd.read_csv(f'image_links/{entity}_entity_value_links.csv')

    # Prepare lists to store results
    results = []

    for _, row in df.iterrows():
        image_link = row['image_link']
        actual_value = row['entity_value']
        predicted_value = get_prediction(image_link, entity)

        # Store result
        results.append({
            'image_link': image_link,
            'actual_value': actual_value,
            'predicted_value': predicted_value
        })

    # Save results to CSV
    result_df = pd.DataFrame(results)
    result_df.to_csv(f'image_links/{entity}_results.csv', index=False)

/home/sukhvansh/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 22 files: 100%|██████████| 22/22 [00:00<00:00, 117998.32it/s]
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


FlashAttention2 is not installed.
An error occurred while creating the pipeline: invalid load key, '\xd5'.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:53                                                                                   │
│                                                                                                  │
│   50 )                                                                                           │
│   51                                                                                             │
│   52 try:                                                                                        │
│ ❱ 53 │   pipe = pipeline(model, backend_config=config)                                           │
│   54 except Exception as e:                                                                      │
│   55 │   print(f"An error occurred while creating the pipeline: {e}")                            │
│   56 │   raise                                                                                   │
│                                                                                                  │
│ /home/sukhvansh/.local/lib/python3.8/site-packages/lmdeploy/api.py:77 in pipeline                │
│                                                                                                  │
│    74 │   │   backend_config) is PytorchEngineConfig else 'turbomind'                            │
│    75 │   logger.info(f'Using {backend} engine')                                                 │
│    76 │                                                                                          │
│ ❱  77 │   return pipeline_class(model_path,                                                      │
│    78 │   │   │   │   │   │     backend=backend,                                                 │
│    79 │   │   │   │   │   │     backend_config=backend_config,                                   │
│    80 │   │   │   │   │   │     chat_template_config=chat_template_config,                       │
│                                                                                                  │
│ /home/sukhvansh/.local/lib/python3.8/site-packages/lmdeploy/serve/vl_async_engine.py:24 in       │
│ __init__                                                                                         │
│                                                                                                  │
│    21 │   │   backend_config = kwargs.get('backend_config', None)                                │
│    22 │   │   if kwargs.get('backend', '') == 'pytorch':                                         │
│    23 │   │   │   try_import_deeplink(backend_config.device_type)                                │
│ ❱  24 │   │   self.vl_encoder = ImageEncoder(model_path,                                         │
│    25 │   │   │   │   │   │   │   │   │      vision_config,                                      │
│    26 │   │   │   │   │   │   │   │   │      backend_config=backend_config)                      │
│    27 │   │   super().__init__(model_path, **kwargs)                                             │
│                                                                                                  │
│ /home/sukhvansh/.local/lib/python3.8/site-packages/lmdeploy/vl/engine.py:90 in __init__          │
│                                                                                                  │
│    87 │   │   │   │    vision_config: VisionConfig = None,                                       │
│    88 │   │   │   │    backend_config: Optional[Union[TurbomindEngineConfig,                     │
│    89 │   │   │   │   │   │   │   │   │   │   │   │   PytorchEngineConfig]] = None):             │
│ ❱  90 │   │   self.model = load_vl_model(model_path, backend_config=backend_config)              │
│    91 │   │   if vision_config is None:                                                          │
│    92 │   │   │   vision_config = VisionConfig()           

In [ ]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


1
0
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
from google.colab import drive
drive.mount('/content/drive')
drive.mount("/content/Computers")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/Computers


In [8]:
!ls
!cd /content/drive/MyDrive/Amazon_ml_challange


drive  sample_data


In [3]:
!pip install lmdeploy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.8 MB/s eta 0:00:00

In [4]:
!pip install timm
!pip install flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 73.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187290390 sha256=c50f5de67a8b75bcfcf4a34257622475f9b56d59da58a539476a21db9d5b9867
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [11]:
%cd /content/drive/MyDrive/Amazon_ml_challange

/content/drive/.shortcut-targets-by-id/15OPGZDb--VGn3czrClhpba-tiPY8-g26/Amazon_ml_challange


In [12]:
!cd /content/drive/MyDrive/Amazon_ml_challange
!ls
!chmod -R 777 model_offload  # Gives full read/write/execute permissions


 Bridging-Text-Spotting			    model_offload
 car.jpg				    new_image.jpg
 clipEmbeddings.py			    output
 clusteering.py				    paligemma_fine_tune.py
 clustered_images			    paligemma.ipynb
 dataset				    paligemma_test.py
 deep-text-recognition-benchmark	    prismer
 download.py				    __pycache__
 entity_csvs				    README.md
 file_maker.py				   'sample_code InternVLM2.py'
 final_inference.py			    sample_code.py
 geminiVisionAPI.ipynb			    seleniumScriptInternVLM2.py
 geminiVisionAPI.py			    seleniumScriptInterVLM2.py
 image_links				    smarter
 images					    src
 inference_results.csv			   'student_resource 3'
 Inference_with_TrOCR_+_Gradio_demo.ipynb   System.txt
 internVL_API.py			    test.csv
 internVLM2.ipynb			    testImages
 InternVLM2.py				    test.ipynb
 kosmos.ipynb				    train.csv
 message.py				    transformed_data.jsonl


In [13]:
import pandas as pd
import os
from lmdeploy import pipeline, TurbomindEngineConfig
from lmdeploy.vl import load_image

path = "testImages"

# Function to get predictions
def get_prediction(image_name, entity_name, pipe):
    modified_file_path = image_name.split('/')[-1]
    image = load_image(f'{path}/{modified_file_path}')
    query = f"""What is the {entity_name} of the product? Only return the numerical value with the unit. Follow the following return format:
    Output only the extracted value in the format "x unit," where x is a float and unit is from the allowed list below.
    - Do not include any additional text except for units.
    Example: Return "12.0 inch" or "500 gram" (without additional phrases or short forms).
    """
    response = pipe((query, image))
    return response.text.strip()

# Process each entity-specific CSV file
entity_files = [r'width.csv']
i = 1
for entity_file in entity_files:
    # Extract entity_name from the filename
    entity_name = os.path.splitext(entity_file)[0]

    # Reload the model and configuration for each entity_name
    model = 'OpenGVLab/InternVL2-4B'
    offload_folder = 'model_offload'
    os.makedirs(offload_folder, exist_ok=True)

    config = TurbomindEngineConfig(
        session_len=8192,
        temperature=0.7,
        system_prompt=(
            """Task: Extract relevant information from the image based on the following:

    1. Data Extraction: Extract width, height, depth, item_weight, item_volume, wattage, voltage, or maximum_weight_recommendation using OCR and visual cues.

    2. Return Format:
    - Output only the extracted value in the format "x unit," where x is a float and unit is from the allowed list below.
    - Do not include any additional text like "The prediction is."
    - Example: Return "12.0 inch" or "500 gram" (without additional phrases or short forms).
    - Do not use commas for large numbers. For example, return "10000.0" instead of "10,000."

    3. Allowed Units:
    {
        'width': ['centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'],
        'depth': ['centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'],
        'height': ['centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'],
        'voltage': ['kilovolt', 'millivolt', 'volt'],
        'wattage': ['kilowatt', 'watt'],
        'item_volume': ['centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                        'imperial gallon', 'litre', 'millilitre', 'pint', 'quart']
    }

    4. Examples:
    - Image shows "Height: 12 inches" → Return: "12.0 inch"
    - g to gram, W to Watt
    - No relevant data → Return: " "
    """),
        offload_folder=offload_folder
    )

    # Load the model and pipeline
    try:
        pipe = pipeline(model, backend_config=config, offload_folder=offload_folder)
    except Exception as e:
        print(f"An error occurred while creating the pipeline: {e}")
        continue

    # Load the CSV file for the current entity_name
    entity_df = pd.read_csv(os.path.join('entity_csvs', entity_file), index_col=0)  # Use the original index

    # Prepare lists to store results
    results = []

    # Iterate over each row in the filtered dataframe
    for _, row in entity_df.iterrows():
        original_index = row.name  # Original index from the DataFrame
        image_link = row['image_link']

        # Get prediction for the current image and entity
        predicted_value = get_prediction(image_link, entity_name, pipe)

        # Store result with original row index and predicted value
        results.append({
            'original_index': original_index,
            'image_link': image_link,
            'predicted_value': predicted_value
        })
        print(i)
        i += 1

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)

    # Save results to a CSV file for the current entity_name
    output_filename = f'results_{entity_name}.csv'
    result_df.to_csv(output_filename, index=False)  # Save without adding new index
    print(f"Results for entity '{entity_name}' saved to {output_filename}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

conversation.py:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

...

modeling_internvl_chat.py:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.89k [00:00<?, ?B/s]

Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in Phi3ForCausalLM is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in Phi3Model is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`


2024-09-15 20:08:34,097 - lmdeploy - ERROR - [Errno 5] Input/output error: 'testImages/11TU2clswzL.jpg', image_url=testImages/11TU2clswzL.jpg


RuntimeError: This event loop is already running